# D2E2F: ForSea meeting
(2021-04-29)

### 1) Utveckla simuleringsmodell med Machine Learning
### 2) Använda simuleringsmodell för att optimera thrustrar 

### 3) Analysarbete
#### a) Långtidsanalys: dagar/veckor/månader/år
#### b) Förbättring med "Smart Eco-shipping"
#### c) Korttidsanalys: farygsdynamik under resa

### 4) PhD Chalmers paper
### 5) "Förstå" datan
![](figures/I_want_to_understand.png)

In [ ]:
#%load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)
plt.style.use('presentation')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os
sys.path.append('../')
from src.visualization import visualize, animate
from src.data import get_dataset
from src.data import prepare_dataset
from src.data import trips


import scipy.integrate
import seaborn as sns

In [ ]:
df = get_dataset.get(name='tycho_short_id')

## Plot trips

In [ ]:
visualize.plot_trips(df=df, width=1400, height=800, zoom_start=14, color_key='trip_direction')

## Varför åker man inte raka vägen?

## Trip statistics

In [ ]:
df_stat = get_dataset.trip_statistics()

In [ ]:
df_stat_plot = df_stat.copy()

df_stat_plot['energy per trip [Nm]'] = df_stat_plot['power_em_thruster_total']

df_stat_plot['trip_direction'] = df_stat_plot['trip_direction'].apply(lambda x : 'Helsingør-Helsingborg' if x==0 else 'Helsingborg-Helsingør')

fig = px.scatter(df_stat_plot, x='start_time',y='energy per trip [Nm]', color='trip_direction', width=1500, height=600, 
                color_discrete_sequence=['red','green'])
fig.show()

## Analyzing trips

In [ ]:
fig = px.line(df, x='trip_time', y='sog', color='trip_no', width=1500, height=800)
fig.show()

![](animations/trip_101.gif "101")

## Look at one trip

In [ ]:
trips = df.groupby(by='trip_no')
trip_no = 129
trip = trips.get_group(trip_no)

In [ ]:
visualize.plot_map(df=trip, width=1400, height=800, zoom_start=14, color_key='trip_direction')

In [ ]:
trip[[f'sin_pm{i}' for i in range(1,5)]].isnull().any()

In [ ]:
trip[['cog','heading','sog']].isnull().any()

In [ ]:
animate.widget(trip=trip)

## Is there a more optimal thruster operation?

![](https://memegenerator.net/img/images/15921176.jpg)

## Thruster positions?

Annas definition:

    Thruster 1 – NV
    Thruster 2 – SV
    Thruster 3 – NE
    Thruster 4 - SE


In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==0)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()




fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row, positions=['NV', 'SV', 'NE', 'SE'])
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

In [ ]:
animate.widget(trip=trip, positions=['NV', 'SV', 'NE', 'SE'])

## Transverse force for thrusters during one trip

In [ ]:
sin_keys = ['sin_pm%i' % n for n in range(1,5)]
cos_keys = ['cos_pm%i' % n for n in range(1,5)]
power_keys = ['power_em_thruster_%i' % n for n in range(1,5)]
columns = sin_keys + cos_keys + power_keys

In [ ]:
g = sns.PairGrid(trip[sin_keys])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False);

## Compare mean power for many trips

In [ ]:
df_mean =  df.groupby('trip_no').mean()

In [ ]:
g = sns.PairGrid(df_mean[power_keys + ['reversing']])
g.map_upper(sns.scatterplot)
#g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3, legend=False);

In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==1)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()

fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row)
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

In [ ]:
df_no_reverse = df.groupby('trip_no').filter(lambda x : x.iloc[0]['reversing']==0)
groups = df_no_reverse.groupby('trip_no')
trip_no_reverse =groups.get_group(list(groups.groups.keys())[0])
trip_ = animate.normalize_power(trip=trip_no_reverse)
row = trip_.iloc[500].copy()

fig,ax=plt.subplots()
fig.set_size_inches(6,6)
animate.plot_thrusters(ax=ax, row=row)
ax.set_title(f'reversing:{row["reversing"]}, trip_direction: {row["trip_direction"]}');

## Fartygsdata?
![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f1/20190531_Fersea_Tycho_Brahe_Helsingor_0120_%2847981263576%29.jpg/335px-20190531_Fersea_Tycho_Brahe_Helsingor_0120_%2847981263576%29.jpg)
* GA?
* Max effekt på thrustrar?

## Nästa möte...